# Merging der unbereinigten Datensätze

**Libraries**<br>
pandas:     Datenverarbeitung<br>
os:         Betriebsystem-Funktionen für relative Pfadreferenzierung<br>

In [ ]:
import pandas as pd
import os

**Dateipfade**<br>
Wetterdaten Braunschweig<br>
Haushaltslasten Braunschweig<br>

In [29]:
dirname = os.path.abspath('')

path_precipitation =    os.path.join(dirname, '2019_Braunschweig_Niederschlag_15min_UTC.csv')
path_solar =            os.path.join(dirname, '2019_Braunschweig_Solar_15min_UTC.csv')
path_wind =             os.path.join(dirname, '2019_Braunschweig_Wind_15min_UTC.csv')
path_air =              os.path.join(dirname, '2019_Braunschweig_Luft_15min_UTC.csv')

path_building1 =        os.path.join(dirname, 'Rohdaten_Last_Braunschweig\Building1.csv')
path_building2 =        os.path.join(dirname, 'Rohdaten_Last_Braunschweig\Building2.csv')
path_building3 =        os.path.join(dirname, 'Rohdaten_Last_Braunschweig\Building3.csv')
path_building4 =        os.path.join(dirname, 'Rohdaten_Last_Braunschweig\Building4.csv')
path_building5 =        os.path.join(dirname, 'Rohdaten_Last_Braunschweig\Building5.csv')

**Einlesen und Merging der Wetterdaten (Predictors)**

In [30]:
data_precipitation =    pd.read_csv(path_precipitation)
data_solar =            pd.read_csv(path_solar)
data_wind =             pd.read_csv(path_wind)
data_air =              pd.read_csv(path_air)

**Merging der Wetterdaten**<br>
Die einzelnen dataframes werden auf Basis der timestamps (MESS_DATUM) zusammengefasst. Dafür werden die strings der MESS_DATUM-Spalte in ein Datum konvertiert. Der Parameter utc=True fügt dem Datum einen Zeitzonenstempel hinzu (+00:00, default), dieser wird später für für das Merging in Stundenwerte benötigt.

In [31]:
# Liste der dataframes
dfs_list = [data_precipitation, data_solar, data_wind, data_air]

# 'timestamp' Konvertierung
for i, df in enumerate(dfs_list):
    dfs_list[i]['MESS_DATUM'] = pd.to_datetime(df['MESS_DATUM'], utc=True)

# Merging auf Basis des timestamps
merged_df = dfs_list[0]
for i in range(1, len(dfs_list)):
    merged_df = pd.merge_asof(merged_df, dfs_list[i], on='MESS_DATUM')

merged_df.to_csv('2019_Merged_Data_15min.csv', index=False) #index=FALSE for not including row indices
print(merged_df)

                     MESS_DATUM  RWS_DAU_10  RWS_10  DS_10  GS_10  SD_10  \
0     2019-01-01 00:00:00+00:00           8    0.00    0.0    0.0    0.0   
1     2019-01-01 00:10:00+00:00          10    0.00    0.0    0.0    0.0   
2     2019-01-01 00:20:00+00:00          10    0.00    0.0    0.0    0.0   
3     2019-01-01 00:30:00+00:00          10    0.03    0.0    0.0    0.0   
4     2019-01-01 00:40:00+00:00          10    0.01    0.0    0.0    0.0   
...                         ...         ...     ...    ...    ...    ...   
52555 2019-12-31 23:10:00+00:00           0    0.00    0.0    0.0    0.0   
52556 2019-12-31 23:20:00+00:00           0    0.00    0.0    0.0    0.0   
52557 2019-12-31 23:30:00+00:00           0    0.00    0.0    0.0    0.0   
52558 2019-12-31 23:40:00+00:00           0    0.00    0.0    0.0    0.0   
52559 2019-12-31 23:50:00+00:00           0    0.00    0.0    0.0    0.0   

       FF_10  DD_10   PP_10  TT_10  TM5_10  RF_10  
0        4.4    260  1020.7    7.9 

**10-minütigen Wettermessungen werden zu stündlichen zusammengefasst**<br>
Die Wahl von sum() und mean() beim Resampling wird auf Basis der jeweiligen Messgröße getroffen.
Basierend auf den timespamps "MESS_DATUM" werden die dataframes gemerged.

In [32]:
# Dictionary mit den Resampling-Funktionen je Messgröße
aggregation_dict = {
    'RWS_DAU_10':   'sum',      # Regendauer (10-min Messungen)
    'RWS_10':       'sum',      # Regenmenge (Höhe in mm, 10-min Messungen)
    'DS_10':        'sum',      # Diffuse Strahlung (10-min Messungen)
    'GS_10':        'sum',      # Globale Strahlung (10-min Messungen)
    'SD_10':        'sum',      # Sonnenschein-Dauer (10-min Messungen)
    'FF_10':        'mean',     # Durchschn. Windgeschwindigkeit
    'DD_10':        'mean',     # Durchschn. Windrichtung
    'PP_10':        'mean',     # Luftdruck auf Höhe der Messstation
    'TT_10':        'mean',     # Lufttemperatur 2 m über dem Boden
    'TM5_10':       'mean',     # Lufttemperatur 5 cm über dem Boden
    'RF_10':        'mean'      # Relative Luftfeuchtigkeit
}

# Resampling auf Basis des Dictionaries, resample-Funktion benötigt zusätzliche Indexierung
merged_df.set_index('MESS_DATUM', inplace=True)
merged_df = merged_df.resample('1h').agg(aggregation_dict)

# Einfügen der Zeilenindexierung
merged_df = merged_df.reset_index()

# Print DataFrame
print(merged_df)

                    MESS_DATUM  RWS_DAU_10  RWS_10  DS_10  GS_10  SD_10  \
0    2019-01-01 00:00:00+00:00          53    0.04    0.0    0.0    0.0   
1    2019-01-01 01:00:00+00:00          55    0.03    0.0    0.0    0.0   
2    2019-01-01 02:00:00+00:00          40    0.03    0.0    0.0    0.0   
3    2019-01-01 03:00:00+00:00           0    0.00    0.0    0.0    0.0   
4    2019-01-01 04:00:00+00:00          28    0.00    0.0    0.0    0.0   
...                        ...         ...     ...    ...    ...    ...   
8755 2019-12-31 19:00:00+00:00           0    0.00    0.0    0.0    0.0   
8756 2019-12-31 20:00:00+00:00           0    0.00    0.0    0.0    0.0   
8757 2019-12-31 21:00:00+00:00           0    0.00    0.0    0.0    0.0   
8758 2019-12-31 22:00:00+00:00           0    0.00    0.0    0.0    0.0   
8759 2019-12-31 23:00:00+00:00           0    0.00    0.0    0.0    0.0   

         FF_10       DD_10        PP_10     TT_10    TM5_10      RF_10  
0     5.416667  263.333333

**15-minütigen Messungen des Gebäudelast-Datensatzes werden auf stündliche Werte zusammengefasst.**<br>
Addition der vier Messwerte innerhalb einer Stunde.

In [33]:
data_building1 = pd.read_csv(path_building1)
data_building2 = pd.read_csv(path_building2)
data_building3 = pd.read_csv(path_building3)
data_building4 = pd.read_csv(path_building4)
data_building5 = pd.read_csv(path_building5)

data_building_list = [data_building1, data_building2, data_building3, data_building4, data_building5]
# Loop through each data_building DataFrame
for i, df in enumerate(data_building_list):
    # Convert the 'rec_time' column to datetime if it's not already
    df['rec_time'] = pd.to_datetime(df['rec_time'], utc=True)
    # Set the 'rec_time' column as the index
    df.set_index('rec_time', inplace=True)
    df.index += pd.Timedelta(hours=1)
    # Create a DatetimeIndex to enable resampling
    df.index = pd.to_datetime(df.index)
    # Resample the data to 1-hour intervals, maintaining the sum of each hour
    df = df.resample('1h', origin='start').sum()
    # Reset the index to turn the index into a regular column
    df = df.reset_index()
    data_building_list[i] = df

**Merging der Wetter- und Haushaltslastdaten mit anschließender Verkettung**<br>

In [34]:
data_building1 = pd.merge(merged_df, data_building_list[0], left_index=True, right_index=True)
data_building2 = pd.merge(merged_df, data_building_list[1], left_index=True, right_index=True)
data_building3 = pd.merge(merged_df, data_building_list[2], left_index=True, right_index=True)
data_building4 = pd.merge(merged_df, data_building_list[3], left_index=True, right_index=True)
data_building5 = pd.merge(merged_df, data_building_list[4], left_index=True, right_index=True)

df = pd.concat([data_building1, data_building2, data_building3, data_building4, data_building5], ignore_index=True)

# 'rec_time' wird nicht mehr benötigt, da die Wetterdaten mit 'MESS_DATUM' einen timestamp mitbringen
df = df.drop('rec_time', axis=1)

print(df)

df.to_csv('2019_Merged_Data_1h.csv', index=False) #index=FALSE for not including row indices

                     MESS_DATUM  RWS_DAU_10  RWS_10  DS_10  GS_10  SD_10  \
0     2019-01-01 00:00:00+00:00          53    0.04    0.0    0.0    0.0   
1     2019-01-01 01:00:00+00:00          55    0.03    0.0    0.0    0.0   
2     2019-01-01 02:00:00+00:00          40    0.03    0.0    0.0    0.0   
3     2019-01-01 03:00:00+00:00           0    0.00    0.0    0.0    0.0   
4     2019-01-01 04:00:00+00:00          28    0.00    0.0    0.0    0.0   
...                         ...         ...     ...    ...    ...    ...   
43795 2019-12-31 19:00:00+00:00           0    0.00    0.0    0.0    0.0   
43796 2019-12-31 20:00:00+00:00           0    0.00    0.0    0.0    0.0   
43797 2019-12-31 21:00:00+00:00           0    0.00    0.0    0.0    0.0   
43798 2019-12-31 22:00:00+00:00           0    0.00    0.0    0.0    0.0   
43799 2019-12-31 23:00:00+00:00           0    0.00    0.0    0.0    0.0   

          FF_10       DD_10        PP_10     TT_10    TM5_10      RF_10  \
0      5.416